In [1]:
from imutils import contours
import numpy as np
import argparse
import cv2
import myutils

In [116]:
template='ocr_a_reference.png'
image_path='images/credit_card_01.png'

In [13]:
# set card type
FIRST_NUMBER = {
	"3": "American Express",
	"4": "Visa",
	"5": "MasterCard",
	"6": "Discover Card"
}

In [14]:
# display
def cv_show(name,img):
	cv2.imshow(name, img)
	cv2.waitKey(0)
	cv2.destroyAllWindows()

# Template processing

In [61]:
# load a template image
img = cv2.imread(template)
cv_show('img',img)
ref = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv_show('ref',ref)

In [69]:
ref = cv2.threshold(img,10,255,cv2.THRESH_BINARY_INV)[1]
cv_show('ref',ref)

In [74]:
ref=ref[:,:,0] #hjp

In [85]:
refCnts,hierarchy = cv2.findContours(ref.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(img,refCnts,-1,(0,255,0),3) 

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [86]:
cv_show('img',img)

In [87]:
print(np.array(refCnts).shape)

(10,)


In [95]:
refCnts = myutils.sort_contours(refCnts, method="left-to-right")[0] #sort, from left to right, from up to down 
digits = {}

In [166]:
# interate over every contours
for (i, c) in enumerate(refCnts):
    (x,y,w,h) = cv2.boundingRect(c)
    roi = ref[y:y+h, x:x+w]
    roi = cv2.resize(roi, (57,88))
    digits[i] = roi

# Image Processing
find the position to load

In [167]:
# initialize convolution kernel
rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 3))
sqKernel=cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

In [168]:
image = cv2.imread(image_path)
cv_show('image',image)
h,w=image.shape[:2]
image= cv2.resize(image,(300,int(h*300/w))) #wide 300
gray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv_show('gray',gray)

In [169]:
tophat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, rectKernel) 
cv_show('tophat',tophat) 

In [170]:
gradX = cv2.Sobel(tophat, ddepth=cv2.CV_32F, dx=1, dy=0, #ksize=-1 equals 3*3
	ksize=-1)
gradX = np.absolute(gradX)
(minVal, maxVal) = (np.min(gradX), np.max(gradX))
gradX = (255 * ((gradX - minVal) / (maxVal - minVal)))
gradX = gradX.astype("uint8")

print (np.array(gradX).shape)
cv_show('gradX',gradX)

(189, 300)


In [171]:
#Perform morphological closing
gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKernel) 
cv_show('gradX',gradX)

In [172]:
#THRESH_OTSU will automatically find the optimal value
thresh = cv2.threshold(gradX, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1] 
cv_show('thresh',thresh)

In [173]:
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, sqKernel) #one more closing
cv_show('thresh',thresh)

In [174]:
# get contours
threshCnts, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

In [175]:
cnts = threshCnts
cur_img = image.copy()
cv2.drawContours(cur_img,cnts,-1,(0,0,255),3) 
cv_show('img',cur_img)
locs = []

# Find the postion of digits based on Contours

In [176]:
# iterate over all contours
for (i, c) in enumerate(cnts):
	# Get rectangles
	(x, y, w, h) = cv2.boundingRect(c)
	ar = w / float(h)

	if ar > 2.5 and ar < 4.0:

		if (w > 40 and w < 55) and (h > 10 and h < 20):

			locs.append((x, y, w, h))

In [177]:
# sort the suitable contours
locs = sorted(locs, key=lambda x:x[0])
output = []

In [178]:
for (i, (gX, gY, gW, gH)) in enumerate(locs):
    # initialize the list of group digits
    groupOutput = []
    group = gray[gY - 5:gY + gH + 5, gX - 5:gX + gW + 5] # set a larger range
    cv_show('group',group)
    group = cv2.threshold(group, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    cv_show('group',group)
    
    digitCnts, hierarchy = cv2.findContours(group.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    digitCnts = contours.sort_contours(digitCnts, method='left-to-right')[0]
    
    for c in digitCnts:
        # find the contour of current digit and resize it
        (x,y,w,h) = cv2.boundingRect(c)
        roi = group[y:y+h, x:x+w]
        roi = cv2.resize(roi,(57,88))
        cv_show('roi',roi)
        
        scores = []
        
        for (digit, digitROI) in digits.items():
            result = cv2.matchTemplate(roi, digitROI,cv2.TM_CCOEFF)
            _,score, _, _ = cv2.minMaxLoc(result)
            scores.append(score)
        groupOutput.append(str(np.argmax(scores)))
    
    # plot
    cv2.rectangle(image, (gX-5, gY-5),(gX + gW + 5, gY + gH + 5), (0, 0, 255), 1)
    cv2.putText(image, "".join(groupOutput), (gX,gY-15), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,0,255), 2)
    
    # get the result
    output.extend(groupOutput)       

In [180]:
print("Credit Card Type: {}".format(FIRST_NUMBER[output[0]]))
print("Credit Card #: {}".format("".join(output)))
cv2.imshow("Image", image)
cv2.waitKey(0)

Credit Card Type: Visa
Credit Card #: 4000123456789010


-1